### Imports

In [1]:
from dataclasses import dataclass
from pathlib import Path
from Phishing_Detector.utils import *
from Phishing_Detector.constants import *
from sklearn.impute import SimpleImputer
from urllib import request
from scipy.io import arff
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
os.chdir('../')
#os.chdir('../DS_Projects/Phishing-Domain-Detection/')
os.getcwd()

'c:\\DS_Projects\\Phishing-Domain-Detection'

### Config Entity

In [13]:
@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_validation_report: Path
    processed_data_file_path: Path
    transformed_data_dir_path: Path
    transformed_data_file_path: Path
    train_test_data_dir_path: Path
    x_train_file_path: Path
    x_test_file_path: Path
    y_train_file_path: Path
    y_test_file_path: Path
    param_imputer_strategy: str
    param_test_size: float
    param_random_state: int
    param_target: str

### Configuration

In [14]:
class ConfigurationManager:

    def __init__(
            self, 
            config_path = CONFIG_FILE_PATH,
            params_path = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)
        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        
        config = self.config.data_transformation
        params = self.params
        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir = config.root_dir,
            data_validation_report = config.data_validation_report,
            processed_data_file_path = config.processed_data_file_path,
            transformed_data_dir_path = config.transformed_data_dir_path,
            transformed_data_file_path = config.transformed_data_file_path,
            train_test_data_dir_path = config.train_test_data_dir_path,
            x_train_file_path = config.x_train_file_path,
            x_test_file_path = config.x_test_file_path,
            y_train_file_path = config.y_train_file_path,
            y_test_file_path = config.y_test_file_path,
            param_imputer_strategy = params.IMPUTER_STRATEGY,
            param_test_size = params.TEST_SIZE,
            param_random_state = params.RANDOM_STATE,
            param_target = params.TARGET
        )

        return data_transformation_config



In [18]:
class DataTransformation:
    
    def __init__(self, config:DataTransformationConfig):
        print(f"{'>'*10} Starting Data Transformation Stage! {'<'*10}")
        
        print(f"Getting Configuration for Data Transformation Stage...")
        self.config = config
        print(f"Configurations : {config}")

        print(f"Creating required directories...")
        create_directories([self.config.transformed_data_dir_path])
        create_directories([self.config.train_test_data_dir_path])

    def get_validation_status(self) -> bool:
        print(f"Loading data validation report...")
        validation_report = load_json(Path(self.config.data_validation_report))
        final_validation_status = validation_report['final_validation_status']
        print(f"Data Validation Status : {final_validation_status}")
        return final_validation_status

    def transform_data(self):
        if self.get_validation_status():
            print(f"Reading the validated data...")
            data = pd.read_csv(self.config.processed_data_file_path)

            print(f"Filling null values using SimpleImputer(strategy = {self.config.param_imputer_strategy})...")
            imputer = SimpleImputer(strategy=self.config.param_imputer_strategy)
            imputer.fit(data)
            print(f"Null Values Imputed Successfully...")
            imputed_data = imputer.transform(data)
            imputed_df = pd.DataFrame(imputed_data, columns=data.columns)
            imputed_df.to_csv(self.config.transformed_data_file_path, index = False)
            print(f"Transformed Data Saved at {self.config.transformed_data_file_path}...")
        else:
            print(f'Data is not validated - Validation Status = {self.get_validation_status()}')

    def split_data(self):
        print(f"Splitting the transformed data into train and test splits...")

        data = pd.read_csv(self.config.transformed_data_file_path)
        X = data.drop(self.config.param_target, axis = 1)
        Y = data[self.config.param_target]
        x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size= self.config.param_test_size, 
                                                            random_state= self.config.param_random_state)
        print(f"Train and test splits created successfully...")

        x_train.to_csv(self.config.x_train_file_path, index = False)
        print(f"x_train saved at {self.config.x_train_file_path}")

        x_test.to_csv(self.config.x_test_file_path, index = False)
        print(f"x_test saved at {self.config.x_test_file_path}")

        y_train.to_csv(self.config.y_train_file_path, index = False)
        print(f"y_train saved at {self.config.y_train_file_path}")

        y_test.to_csv(self.config.y_test_file_path, index = False)
        print(f"y_test saved at {self.config.y_test_file_path}")

        print(f"{'>'*10} Data Transformation Stage Completed Successfully! {'<'*10}")

In [19]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config = data_transformation_config)
    data_transformation.transform_data()
    data_transformation.split_data()
except Exception as e:
    raise e

>>>>>>>>>> Starting Data Transformation Stage! <<<<<<<<<<
Getting Configuration for Data Transformation Stage...
Configurations : DataTransformationConfig(root_dir='artifacts/data_transformation', data_validation_report='artifacts/data_validation/validation_report/report.json', processed_data_file_path='artifacts/data_ingestion/processed_data/data.csv', transformed_data_dir_path='artifacts/data_transformation/transformed_data', transformed_data_file_path='artifacts/data_transformation/transformed_data/transformed_data.csv', train_test_data_dir_path='artifacts/data_transformation/train_test_data', x_train_file_path='artifacts/data_transformation/train_test_data/x_train.csv', x_test_file_path='artifacts/data_transformation/train_test_data/x_test.csv', y_train_file_path='artifacts/data_transformation/train_test_data/y_train.csv', y_test_file_path='artifacts/data_transformation/train_test_data/y_test.csv', param_imputer_strategy='most_frequent', param_test_size=0.25, param_random_state=42,